<a href="https://colab.research.google.com/github/TUIlmenauAMS/Videocoding/blob/main/DCVC_FM_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Colab Example to run the DCVC Video Coder

Needs to be run with GPU in the runtime type.

In [1]:
# Download and prepare the test video data
! mkdir data
%cd data
! wget -O HoneyBee_1920x1080_120fps_420_8bit_YUV_RAW.7z https://ultravideo.fi/video/HoneyBee_1920x1080_120fps_420_8bit_YUV_RAW.7z
! apt-get install p7zip-full
! 7z x HoneyBee_1920x1080_120fps_420_8bit_YUV_RAW.7z
! mkdir HoneyBee_1920x1080
! ffmpeg -pix_fmt yuv420p -s 1920x1080 -i HoneyBee_1920x1080_120fps_420_8bit_YUV.yuv   -f image2 HoneyBee_1920x1080/im%05d.png
! rm HoneyBee_1920x1080_120fps_420_8bit_YUV_RAW.7z
%cd ..

/content/data
--2024-07-13 06:21:35--  https://ultravideo.fi/video/HoneyBee_1920x1080_120fps_420_8bit_YUV_RAW.7z
Resolving ultravideo.fi (ultravideo.fi)... 130.230.203.118
Connecting to ultravideo.fi (ultravideo.fi)|130.230.203.118|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 906770507 (865M)
Saving to: ‘HoneyBee_1920x1080_120fps_420_8bit_YUV_RAW.7z’

HoneyBee_1920x1080_ 100%[===================>] 864.76M  18.3MB/s    in 50s     

2024-07-13 06:22:25 (17.4 MB/s) - ‘HoneyBee_1920x1080_120fps_420_8bit_YUV_RAW.7z’ saved [906770507/906770507]

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
p7zip-full is already the newest version (16.02+dfsg-8).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.

7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,12 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (50657),ASM,AES-NI)

Scanning

In [2]:
# Clone the DCVC Git repository
! git clone https://github.com/microsoft/DCVC.git
%cd /content/DCVC/DCVC-FM

Cloning into 'DCVC'...
remote: Enumerating objects: 355, done.
remote: Counting objects: 100% (122/122), done.
remote: Compressing objects: 100% (87/87), done.
remote: Total 355 (delta 48), reused 77 (delta 34), pack-reused 233
Receiving objects: 100% (355/355), 6.43 MiB | 12.24 MiB/s, done.
Resolving deltas: 100% (129/129), done.
/content/DCVC/DCVC-FM


In [3]:
# Edit the RGB dataset file to only include the video data
%%writefile dataset_config_example_rgb.json
{
    "root_path": "/content/",
    "test_classes": {
        "UVG": {
            "test": 1,
            "base_path": "data",
            "src_type": "png",
            "sequences": {
                "HoneyBee_1920x1080":      {"width": 1920, "height": 1080, "frames": 96, "intra_period": 96}
            }
        }
    }
}

Overwriting dataset_config_example_rgb.json


In [4]:
# Edit the YUV dataset file to only include the video data
%%writefile dataset_config_example_yuv420.json
{
    "root_path": "/content/",
    "test_classes": {
        "UVG": {
            "test": 1,
            "base_path": "data",
            "src_type": "yuv420",
            "sequences": {
                "HoneyBee_1920x1080_120fps_420_8bit_YUV":      {"width": 1920, "height": 1080, "frames": 600, "intra_period": -1}
            }
        }
    }
}

Overwriting dataset_config_example_yuv420.json


In [5]:
# Build the arithmetic codec
%cd src
! mkdir build
%cd build
! cmake ../cpp -DCMAKE_BUILD_TYPE=Release
! make -j
%cd ..
%cd ..

! apt-get install ninja-build
%cd ./src/models/extensions/
! python setup.py build_ext --inplace
%cd /content/DCVC/DCVC-FM

# Download checkpoints
%cd checkpoints/
! python download.py
%cd ..

/content/DCVC/DCVC-FM/src
/content/DCVC/DCVC-FM/src/build
-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Configuring done (0.0s)
-- Generating done (0.0s)
-- Build files have been written to: /content/DCVC/DCVC-FM/src/build/3rdparty/pybind11/pybind11-download
[ 11%] Creating directories for 'pybind11'
[ 22%] Performing download step (git clone) for 'pybind11'
Cloning into 'pybind11-src'...
HEAD is now at 5b0a6fc chore: bump version to 3.10.4
[ 33%] No update step for 'pybind11'
[ 44%] No patch step for 'pybind11'
[ 55%] No configure 

In [6]:
# Install required packages
! python -m pip install -r /content/DCVC/DCVC-FM/requirements.txt
! pip install torch torchvision torchaudio

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [7]:
%cd /content/DCVC/DCVC-FM
! python test_video.py --model_path_i checkpoints/cvpr2024_image.pth.tar --model_path_p checkpoints/cvpr2024_video.pth.tar --rate_num 4 --test_config dataset_config_example_rgb.json --cuda 1 --worker 1 --write_stream 1 --output_path output.json --force_intra_period 9999 --force_frame_num 96 --save_decoded_frame 1

/content/DCVC/DCVC-FM
testing 4 rates, using q_indexes: 0, 21, 42, 63, 
  0% 0/4 [00:00<?, ?it/s]created folder: out_bin/UVG
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
100% 4/4 [12:54<00:00, 193.59s/it]
Test finished
Tested 384 frames from 1 sequences
Total elapsed time: 12.9 min
